# LAB | Hyperparameter Tuning

**Load the data**

Finally step in order to maximize the performance on your Spaceship Titanic model.

The data can be found here:

https://raw.githubusercontent.com/data-bootcamp-v4/data/main/spaceship_titanic.csv

Metadata

https://github.com/data-bootcamp-v4/data/blob/main/spaceship_titanic.md

So far we've been training and evaluating models with default values for hyperparameters.

Today we will perform the same feature engineering as before, and then compare the best working models you got so far, but now fine tuning it's hyperparameters.

In [1]:
#Libraries
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.tree import DecisionTreeClassifier

from sklearn.preprocessing import MinMaxScaler, StandardScaler
from sklearn.metrics import r2_score, mean_absolute_error, mean_squared_error

In [2]:
spaceship = pd.read_csv("https://raw.githubusercontent.com/data-bootcamp-v4/data/main/spaceship_titanic.csv")
spaceship.head()

,PassengerId,HomePlanet,CryoSleep,Cabin,Destination,Age,VIP,RoomService,FoodCourt,ShoppingMall,Spa,VRDeck,Name,Transported
0,0001_01,Europa,False,B/0/P,TRAPPIST-1e,39.0,False,0.0,0.0,0.0,0.0,0.0,Maham Ofracculy,False
1,0002_01,Earth,False,F/0/S,TRAPPIST-1e,24.0,False,109.0,9.0,25.0,549.0,44.0,Juanna Vines,True
2,0003_01,Europa,False,A/0/S,TRAPPIST-1e,58.0,True,43.0,3576.0,0.0,6715.0,49.0,Altark Susent,False
3,0003_02,Europa,False,A/0/S,TRAPPIST-1e,33.0,False,0.0,1283.0,371.0,3329.0,193.0,Solam Susent,False
4,0004_01,Earth,False,F/1/S,TRAPPIST-1e,16.0,False,303.0,70.0,151.0,565.0,2.0,Willy Santantines,True


Now perform the same as before:
- Feature Scaling
- Feature Selection


In [3]:
spaceship.dropna(inplace= True)
spaceship['Cabin'] = spaceship['Cabin'].str.split('/').str[0]
spaceship =spaceship.drop(columns = ["PassengerId","Name"])
spaceship_cat = spaceship[["HomePlanet", "CryoSleep","Destination","Cabin","VIP"]]
num_f = spaceship.drop(columns=["HomePlanet", "CryoSleep","Destination","Cabin","VIP","Transported"])
non_numerical = pd.get_dummies(spaceship_cat, drop_first = True).astype(int)

In [4]:
features = pd.concat([num_f,non_numerical], axis = 1)
target = spaceship["Transported"]
X_train, X_test, y_train, y_test = train_test_split(features, target, test_size = 0.20, random_state=0)
normalizer =MinMaxScaler()
normalizer.fit(X_train)
X_train_norm = normalizer.transform(X_train)
X_test_norm = normalizer.transform(X_test)

- Now let's use the best model we got so far in order to see how it can improve when we fine tune it's hyperparameters.

In [5]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import RandomizedSearchCV

In [6]:
from sklearn.tree import DecisionTreeRegressor

In [7]:
grid = {"n_estimators": [10,20,30,40], "min_samples_split": [20, 40, 60, 80]}
rf_reg = RandomForestRegressor(DecisionTreeRegressor())

In [8]:
model = GridSearchCV(estimator = rf_reg, param_grid = grid, cv=5)

In [9]:
model.fit(X_train_norm, y_train)

GridSearchCV(cv=5,
             estimator=RandomForestRegressor(n_estimators=DecisionTreeRegressor()),
             param_grid={'min_samples_split': [20, 40, 60, 80],
                         'n_estimators': [10, 20, 30, 40]})

In [10]:
model.best_params_

{'min_samples_split': 40, 'n_estimators': 30}

In [11]:
best_model = model.best_estimator_


In [12]:
best_model

RandomForestRegressor(min_samples_split=40, n_estimators=30)

In [13]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import BaggingClassifier, RandomForestClassifier,AdaBoostClassifier
rf_Classifier = RandomForestClassifier(min_samples_split=40, n_estimators= 100)
rf_Classifier.fit(X_train_norm, y_train)

RandomForestClassifier(min_samples_split=40)

- Evaluate your model

In [14]:
pred = rf_Classifier.predict(X_test_norm)

print("R2 score", rf_Classifier.score(X_test_norm, y_test))

R2 score 0.7859304084720121


**Grid/Random Search**

For this lab we will use Grid Search.

- Define hyperparameters to fine tune.

In [15]:
ada_reg = AdaBoostClassifier(DecisionTreeClassifier())

model = RandomizedSearchCV(estimator = ada_reg, param_distributions = grid, n_iter = 10, cv = 5, n_jobs = -1)


In [16]:
model.fit(X_train_norm,y_train)

ValueError: Invalid parameter 'min_samples_split' for estimator AdaBoostClassifier(estimator=DecisionTreeClassifier(), n_estimators=40). Valid parameters are: ['algorithm', 'base_estimator', 'estimator', 'learning_rate', 'n_estimators', 'random_state'].

- Run Grid Search

In [ ]:
best_model = model.best_estimator_

- Evaluate your model

In [17]:
pred = best_model.predict(X_test_norm)

print("MAE", mean_absolute_error(pred, y_test))
print("RMSE", mean_squared_error(pred, y_test, squared=False))
print("R2 score", best_model.score(X_test_norm, y_test))

MAE 0.2747279891625869
RMSE 0.3790646138204307
R2 score 0.42524007419667087
